In [ ]:
import re, json
import pandas as pd
import numpy as np
import os

# pip install tabula-py
import tabula

# Get the directory where this notebook is located
script_dir = os.getcwd()
PDF = os.path.join(script_dir, "data", "01_Scientific assessment of ozone depletion 2022.pdf")

def to_float(x):
    if pd.isna(x):
        return np.nan
    s = str(x).strip().replace("–", "-").replace("−", "-")
    m = re.search(r"(-?\d+(?:\.\d+)?)", s)
    return float(m.group(1)) if m else np.nan

def parse_lifetime_years(x):
    if pd.isna(x):
        return np.nan
    s = str(x).strip()
    if not s:
        return np.nan
    s = s.replace("–", "-").replace("−", "-")
    m = re.search(r"(\d+(?:\.\d+)?)", s)
    if not m:
        return np.nan
    val = float(m.group(1))
    s_low = s.lower()
    if "day" in s_low:
        return val / 365.25
    if "month" in s_low:
        return val / 12.0
    return val  # assume years

# Table A-5 spans a bunch of pages in the WMO PDF.
# Start with 458–488 (1-indexed) and adjust if needed.
PAGES = "458-493"

dfs = tabula.read_pdf(
    PDF,
    pages=PAGES,
    multiple_tables=True,
    lattice=True,
    guess=True,
    java_options=["-Djava.awt.headless=true"],
)

paired = [
    pd.concat([dfs[i].reset_index(drop=True), dfs[i+1].reset_index(drop=True)], axis=1)
    for i in range(0, len(dfs), 2)
]

def make_unique_columns(df):
    cols = df.columns.astype(str)
    seen = {}
    new_cols = []
    for c in cols:
        if c in seen:
            seen[c] += 1
            new_cols.append(f"{c}.{seen[c]}")
        else:
            seen[c] = 0
            new_cols.append(c)
    df = df.copy()
    df.columns = new_cols
    return df

paired_fixed = [make_unique_columns(df) for df in paired]

long_df = pd.concat(
    [paired_fixed[0]] + [df.iloc[1:] for df in paired_fixed[1:]],
    axis=0,
    ignore_index=True
)


name_col = long_df.columns[0]          # assume first column is the name
other_cols = long_df.columns[1:]

# turn pure-whitespace strings into NaN in the "other" columns
tmp = long_df[other_cols].replace(r"^\s*$", np.nan, regex=True)

# keep rows where at least one "other" value is present
long_df = long_df.loc[~tmp.isna().all(axis=1)].reset_index(drop=True)

# pick the columns you want by 0-based position
keep_idx = [0, 1, 2, 5, 8, 9]

long_df = long_df.iloc[:, keep_idx].copy()

# rename them
long_df.columns = [
    "Name",
    "Formula",
    "CAS",
    "WMO (2022) Total lifetime (years)",
    "ODP",
    "Radiative Efficiency (well mixed) (W m–2 ppb–1)",
]

# -----------------------------
# Parse + clean the final columns
# -----------------------------

# 1) Clean CAS: keep only valid CAS format, else NaN
long_df["CAS"] = long_df["CAS"].astype(str).str.strip()
long_df.loc[~long_df["CAS"].str.match(r"^\d{2,7}-\d{2}-\d$"), "CAS"] = np.nan

# 2) Lifetime -> years (convert days/months to years, etc.)
# Your parse_lifetime_years already handles days/months and assumes years otherwise.
long_df["WMO (2022) Total lifetime (years)"] = long_df["WMO (2022) Total lifetime (years)"].apply(parse_lifetime_years)

# 3) ODP -> float
long_df["ODP"] = long_df["ODP"].apply(to_float)

# 4) Radiative efficiency (well mixed) -> float (optional but usually useful)
long_df["Radiative Efficiency (well mixed) (W m–2 ppb–1)"] = (
    long_df["Radiative Efficiency (well mixed) (W m–2 ppb–1)"].apply(to_float)
)

# (optional) clean text columns
long_df["Name"] = long_df["Name"].astype(str).str.strip().replace({"": np.nan})
long_df["Formula"] = long_df["Formula"].astype(str).str.strip().replace({"": np.nan})








# (optional) reset continuous index
long_df.reset_index(drop=True, inplace=True)


In [68]:
out_path = "output.xlsx"

with pd.ExcelWriter(out_path, engine="openpyxl") as writer:
    long_df.to_excel(writer, sheet_name="data", index=False)


In [58]:
# Parse and clean columns
df["Name"] = long_df["Name"] if "Name" in long_df.columns else np.nan
df["Formula"] = long_df["Formula"] if "Formula" in long_df.columns else np.nan
df["CAS"] = long_df["CAS"].astype(str).str.strip() if "CAS" in long_df.columns else np.nan
df["CAS"] = df["CAS"].where(df["CAS"].str.match(r"^\d{2,7}-\d{2}-\d$"), np.nan) if "CAS" in long_df.columns else np.nan
df["Lifetime (years)"] = df["WMO (2022) Total lifetime (years)"].apply(parse_lifetime_years) if "WMO (2022) Total lifetime (years)" in long_df.columns else np.nan
df["ODP"] = df["ODP"].apply(to_float) if "ODP" in long_df.columns else np.nan
df["Radiative Efficiency (W m–2 ppb–1)"] = df["Radiative Efficiency (well mixed) (W m–2 ppb–1)"].apply(to_float) if "Radiative Efficiency (well mixed) (W m–2 ppb–1)" in df.columns else np.nan

TypeError: 'str' object does not support item assignment

In [ ]:
print(df.to_string())

In [56]:
out_path = "output.xlsx"

with pd.ExcelWriter(out_path, engine="openpyxl") as writer:
    long_df.to_excel(writer, sheet_name="data", index=False)
